In [ ]:
model_name = 'GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct'

from unsloth import FastLanguageModel
train_set = 'id_jvsu_claude'
test_language = 'su'
model_path = f"outputs/{model_name.split('/')[-1]}/{train_set}"
import os
import re
checkpoint = 'checkpoint-1000'
# checkpoint = max(os.listdir(model_path), key=lambda x: int(re.search(r'\d+', x).group()))
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"{model_path}/{checkpoint}",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_..."
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
import pandas as pd
data_path = f"../../dataset/test/test_{test_language}.csv"
def load_test_data(data_path):
    data = pd.read_csv(data_path)
    contexts = []
    endings = []
    EOS_TOKEN = tokenizer.eos_token
    for idx, row in data.iterrows():
        sents = []
        for i in [4,3,2,1]:
            current_sentence = row[f'sentence_{i}']
            current_sentence = current_sentence + '.' if current_sentence[-1] != '.' else current_sentence
            sents.insert(0, current_sentence)
        context = ' '.join(sents) 
        ending = row['correct_ending']
        # ending2 = row['incorrect_ending']
        contexts.append(context)
        endings.append(ending)

    # convert to huggingface dataset
    from datasets import Dataset
    dataset = Dataset.from_dict({'context': contexts, 'ending': endings})
    def formatting_prompts_func(examples):
        contexts  = examples["context"]
        endings = examples["ending"]
        texts = []
        for context, ending in zip(contexts, endings):
            chat = [
                {"role": "user", "content": f"Generate the ending for the following given story context\nStory Context: {context}"},
            ]
            text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            texts.append(text)
        return { "text" : texts, }
    dataset = dataset.map(formatting_prompts_func, batched = True,)
    return dataset
dataset = load_test_data(data_path)
contexts = dataset['text']
correct_endings = dataset['ending']



In [ ]:
from tqdm.auto import tqdm
context = contexts[-25]
inputs = tokenizer(context,return_tensors = "pt").to("cuda")
res = model.generate(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, max_new_tokens=128, use_cache=True, do_sample=False)
predicted_ending = tokenizer.decode(res[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
predicted_ending

In [ ]:
print(context)